In [1]:
import sys
import os
sys.path.append('../src/')
sys.path.append('../')

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

import wavy

/Users/rodrigonader/anaconda3/envs/tf-metal/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DATA_PATH = f"{os.getcwd()}/"
df = pd.read_csv(f"{DATA_PATH}../data/petr4.csv", index_col=0)
df = df.iloc[::-1]

In [4]:
# df = df[['price', 'timeTrade']]
df.reset_index(drop=True, inplace=True)

In [5]:
df.head(2)

,symbol,price,open,high,low,previous,quantityTrades,volumeAmount,volumeFinancier,timeTrade
0,petr4,16.58,16.60,16.60,16.58,16.60,2.0,500.0,9762.0,"Dec 28, 2012 7:29:00 PM"
1,petr4,16.60,16.59,16.63,16.59,16.57,7.0,7300.0,142639.0,"Dec 28, 2012 7:28:00 PM"


In [6]:
df = df[['price']].tail(10000) # ! Just to speed up for tests

In [7]:
df.head(2)

,price
1005428,28.96
1005429,28.96


In [28]:
df

,price
0,0.633210
1,-0.066988
2,0.677272
3,0.764956
4,-0.886906
...,...
995,0.186614
996,-0.351936
997,-0.013651
998,1.103389


In [27]:
import numpy as np
import pandas as pd

# Start with any time series dataframe
df = pd.DataFrame({'price': np.random.randn(1000)}, index=range(1000))

# Create panels. Each panel is composed of a list of frames.
x, y = wavy.create_panels(df, lookback=120, horizon=1)

# x and y are the past and corresponding future data.
# Lookback and Horizon their number of timesteps.
print("Lookback:", len(x[0]), "Horizon:", len(y[0]))

# Plot the target.
y.plot()

# Convert to numpy arrays. Panels contain a train-val-test split by default.
x_train, y_train = x.train.values, y.train.values
x_test, y_test = x.test.values, y.test.values

# Or just instantiate a model.
model = wavy.LinearRegression(x, y)
model.score()

/Users/rodrigonader/Repositories/logspace/wavy/notebooks/../src/wavy/panel.py:70: UserWarning: DataFrame is being sorted!
  warnings.warn("DataFrame is being sorted!")


Lookback: 120 Horizon: 1


In [8]:
%%time
x, y = wavy.create_panels(df, lookback=120, horizon=30)

/Users/rodrigonader/Repositories/logspace/wavy/notebooks/../src/wavy/panel.py:70: UserWarning: DataFrame is being sorted!
  warnings.warn("DataFrame is being sorted!")


CPU times: user 353 ms, sys: 27.4 ms, total: 380 ms
Wall time: 380 ms


In [9]:
x[0].head()

,price
1015427,28.71
1015426,28.65
1015425,28.63
1015424,28.60
1015423,28.62


In [10]:
xmax = x.max()

In [11]:
x[0]

,price
1015427,28.71
1015426,28.65
1015425,28.63
1015424,28.60
1015423,28.62
...,...
1015312,29.16
1015311,29.16
1015310,29.12
1015309,29.09


In [12]:
ymax = y.max()

In [16]:
ymax[0]

,price
1015307,29.12


In [13]:
y[0]

,price
1015307,29.08
1015306,29.08
1015305,29.12
1015304,29.08
1015303,29.07
1015302,29.05
1015301,29.05
1015300,29.05
1015299,29.05
1015298,29.05


In [14]:
ylabels = ymax > xmax # if horizon max is greater than lookback max

ValueError: Can only compare identically-labeled DataFrame objects

In [ ]:
# x here must be converted to pct change, dropna and y should match x

In [ ]:
ylabels.sample(10000, how='spaced').as_dataframe().price.value_counts(normalize=True)

In [ ]:
log = wavy.models.LogisticRegression(x, ymax)
log.fit(epochs=10, verbose=True)
log.score()

In [ ]:
dense = wavy.models.DenseModel(x, ymax, 'classification')
dense.fit(epochs=10, verbose=True)
dense.score()

In [ ]:
conv = wavy.models.ConvModel(x, ymax, 'classification')
conv.fit(epochs=10, verbose=True)
conv.score()

In [ ]:
residuals = conv.residuals()

In [ ]:
len(residuals)

In [ ]:
residuals.sample(10000, how='spaced').plot()